In [93]:
#импортируем нужные либы
import pandahouse as ph
import pandas as pd

In [94]:
#объявляем параметры подключения
connection = dict(database='default',
                  host='https://clickhouse.lab.karpov.courses',
                  user='student',
                  password='dpo_python_2020')

# Задание 2: SQL

## 2.1 Очень усердные ученики

### Условие

Необходимо идентифицировать усердных учеников на образовательных курсах. Эти курсы разделены на уроки, которые, в свою очередь, состоят из маленьких заданий, называемых "горошинами". 

Очень усердным считается ученик, который хотя бы раз за текущий месяц правильно решил 20 таких горошин.

### Входные данные

Для решения задачи у вас есть доступ к таблице `default.peas`, структура которой представлена ниже:

| Название атрибута | Тип атрибута | Смысловое значение               |
|-------------------|--------------|---------------------------------|
| st_id             | int          | ID ученика                      |
| timest            | timestamp    | Время решения карточки           |
| correct           | bool         | Правильно ли решена горошина?   |
| subject           | text         | Дисциплина, в которой находится горошина |

### Задача

Используя данные из таблицы `default.peas`,  необходимо написать оптимальный SQL-запрос, который позволит определить количество учеников, соответствующих критерию "очень усердный ученик".

**Примечание:** Под "усердным учеником" понимается студент, который правильно решил 20 или более задач ("горошин") за текущий месяц.


In [95]:
#пишем запрос, и получаем данные из clickhouse в pandas dataframe
query = """
SELECT
  COUNT(st_id) as hardworking_students_count
FROM
  (
    SELECT
      st_id,
      SUM(correct) AS total_correct
    FROM
      default.peas
    WHERE
      timest :: date BETWEEN '2021-10-01'
      AND '2021-11-01'
    GROUP BY
      st_id
    HAVING
      total_correct >= 20
  )
"""
df = ph.read_clickhouse(query, connection=connection)
df

,hardworking_students_count
0,136


### Ответ：
Количество усердных учеников - 136

## 2.2 Оптимизация воронки

### Условие

Вы работаете с образовательной платформой, которая использует модель trial, позволяя студентам решать до 30 задач бесплатно каждый день. Для получения неограниченного доступа к задачам по специальности, студентам необходимо приобрести полный доступ. Недавно команда провела эксперимент, тестируя новый экран оплаты.

### Входные данные

В вашем распоряжении следующие таблицы:

1. `default.peas` (описание см. в предыдущем задании)
2. `default.studs`:

    | Название атрибута | Тип атрибута | Смысловое значение         |
    |-------------------|--------------|---------------------------|
    | st_id             | int          | ID ученика                |
    | test_grp          | text         | Метка ученика в эксперименте |

3. `default.final_project_check`:

    | Название атрибута | Тип атрибута | Смысловое значение         |
    |-------------------|--------------|---------------------------|
    | st_id             | int          | ID ученика                |
    | sale_time         | timestamp    | Время покупки             |
    | money             | int          | Цена курса                |
    | subject           | text         | Дисциплина                |

### Задача

Цель запроса - анализировать данные и предоставить информацию о различных метриках для групп пользователей. Необходимо рассчитать и выгрузить следующие показатели:

- ARPU (средний доход от пользователя)
- ARPAU (средний доход от активного пользователя)
- CR в покупку (коэффициент конверсии в покупку)
- CR активного пользователя в покупку
- CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике

**Дополнительные детали:**

- ARPU рассчитывается для всех пользователей в экспериментальных группах.
- Активным пользователем считается тот, кто решил более 10 задач правильно в любой дисциплине.
- Активным по математике считается пользователь, решивший 2 или более задачи правильно по математике.

Запрос должен быть оптимизирован для обработки больших объемов данных.


In [97]:
query = """
WITH subquery as 
(
SELECT
    studs.st_id as st_id,
    studs.test_grp as group,
    final_project_check.money as money,
    final_project_check.subject as subject,
    results.count_correct as count_correct,
    results.math_correct as math_correct
FROM 
    studs
LEFT JOIN 
    final_project_check 
    ON 
    studs.st_id = final_project_check.st_id
LEFT JOIN
    (
    SELECT
        st_id,
        SUM(correct) as count_correct,
        SUM(correct) FILTER (WHERE(peas.subject = 'Math')) as math_correct
    FROM peas
    GROUP BY
    st_id
    ) as results
    ON
    studs.st_id = results.st_id
)
SELECT
    group,
    SUM(money) / uniqExact(st_id) as ARPU,
    SUM(money) FILTER (WHERE count_correct > 10) / uniqExact(st_id) FILTER (WHERE count_correct > 10) as ARPAU,
    uniq(st_id) FILTER (WHERE (money > 0))/uniq(st_id)  as CR,
    uniq(st_id) FILTER (WHERE count_correct > 10 and money > 0) / uniq(st_id) FILTER (WHERE count_correct > 10) as CR_active,
    uniq(st_id) FILTER (WHERE subject = 'Math' and math_correct >= 2) / uniq(st_id) FILTER (WHERE math_correct >= 2) as CR_active_math
FROM subquery
GROUP BY group
"""
df = ph.read_clickhouse(query, connection=connection)
df

,group,ARPU,ARPAU,CR,CR_active,CR_active_math
0,control,4540.983607,10393.700787,0.049180,0.110236,0.061224
1,pilot,11508.474576,29739.583333,0.108475,0.260417,0.095238


### Выводы:
- Экспериментальная группа (pilot) показывает значительно лучшие результаты по всем ключевым метрикам по сравнению с контрольной группой.
- Улучшения в ARPU и ARPAU указывают на повышенную монетизацию в экспериментальной группе.
- Значительное увеличение коэффициентов конверсии (CR и CR_active) говорит о том, что нововведения, вероятно, оказали положительное влияние на поведение пользователей, стимулируя их совершать покупки.

#### В целом, данные указывают на успешность проведенного эксперимента. Новый экран оплаты или другие изменения, внесенные в экспериментальную группу, оказались эффективными в увеличении дохода и конверсии пользователей.